### Notebook de test des GNNs sur des Toys Examples : 

In [1]:
import sys
import numpy as np
sys.path.append('../datasets')
from datasets.manager import IMDBBinary, DD
import torch 
import itertools
from tqdm import tqdm

#from utils.utils import visualise_graph, get_adjacency_and_features
from utils.utils import get_adjacency_and_features#, create_batch_from_loader
#from src.gnn import GNNClassifier

from datasets.dataset import *
from train import Train

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import random

import numpy as np
import time
import statistics
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from src.models import GCN, GAT
from datasets.dataloader import DataLoader


In [2]:
IMDB = IMDBBinary()
DD = DD()

#### **Visualisation du dataset**

In [3]:
x_IMBD, y_IMDB = IMDB.dataset.get_data(), IMDB.dataset.get_targets()

In [4]:
node8 = x_IMBD[8]
node8

GraphData(x=torch.Size([18, 1]), edge_index=torch.Size([2, 153]))

`GraphData(x=torch.Size([18, 1]), edge_index=torch.Size([2, 153]))` refers to a graph that has: 
- 18 nodes, each having features of dimension 1
- 153 edges (each between 2 nodes)

In [5]:
#visualise_graph(8, x_IMBD)

The adjency matrix of such a graph is computed by `get_adjacency_and_features` and given by:

In [6]:
adjency_mat, feature_vect = get_adjacency_and_features(node8)
print("Adjency matrix:\n")
print(adjency_mat)
print("Feature vector: \n")
print(feature_vect)

Adjency matrix:

tensor([[0.0556, 0.0572, 0.0589, 0.0609, 0.0630, 0.0654, 0.0680, 0.0711, 0.0745,
         0.0786, 0.0833, 0.0891, 0.0962, 0.1054, 0.1179, 0.1361, 0.1667, 0.2357],
        [0.0000, 0.0588, 0.0606, 0.0626, 0.0648, 0.0673, 0.0700, 0.0731, 0.0767,
         0.0808, 0.0857, 0.0917, 0.0990, 0.1085, 0.1213, 0.1400, 0.1715, 0.2425],
        [0.0000, 0.0000, 0.0625, 0.0645, 0.0668, 0.0693, 0.0722, 0.0754, 0.0791,
         0.0833, 0.0884, 0.0945, 0.1021, 0.1118, 0.1250, 0.1443, 0.1768, 0.2500],
        [0.0000, 0.0000, 0.0000, 0.0667, 0.0690, 0.0716, 0.0745, 0.0778, 0.0816,
         0.0861, 0.0913, 0.0976, 0.1054, 0.1155, 0.1291, 0.1491, 0.1826, 0.2582],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0714, 0.0741, 0.0772, 0.0806, 0.0845,
         0.0891, 0.0945, 0.1010, 0.1091, 0.1195, 0.1336, 0.1543, 0.1890, 0.2673],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0769, 0.0801, 0.0836, 0.0877,
         0.0925, 0.0981, 0.1048, 0.1132, 0.1240, 0.1387, 0.1601, 0.1961, 0.2774],
     

Implémentation de la cross-validation ; 

Les hyperparamètres: 
- Learning Rate
- Nombre de Couche de Convolution
- Dimension de l'espace d'embedding
- Critère d'Early Stopping
- dropout / Weight Decay
- Batch Size
- Nombre d'Epochs

Question de mette ou non la feature du degré des noeuds pour voir l'impact sur la performance

In [17]:
params_list = {"model_type": "GCN",  # "GCN", "GAT", "GIN", "GraphSAGE"
               "n_graph_subsampling": 0, # the number of running graph subsampling each train graph data run subsampling 5 times: increasing graph data 5 times
               "graph_node_subsampling": True, # TRUE: removing node randomly to subsampling and augmentation of graph dataset \n'+
                # FALSE: removing edge randomly to subsampling and augmentation of graph dataset
               "graph_subsampling_rate": 0.2, # graph subsampling rate
               "dataset": "DD", 
               "pooling_type": "mean", 
               "seed": 42,
               "n_folds": 10, 
               "cuda": True, 
               "lr": [0.0001, 0.001, 0.01], 
               "epochs": 1, 
               "weight_decay":5e-4,
               "batch_size": 32, 
               "dropout": 0, # dropout rate of layer
               "num_lay": [1, 2, 3], 
               "num_agg_layer": 2, # the number of graph aggregation layers
               "hidden_agg_lay_size": [16, 32, 64], # size of hidden graph aggregation layer
               "fc_hidden_size": 128, # size of fully-connected layer after readout
               "threads":10, # how many subprocesses to use for data loading
               "random_walk":True,
               "walk_length": 20, # walk length of random walk, 
               "num_walk": 10, # num of random walk
               "p": 0.65, # Possibility to return to the previous vertex, how well you navigate around
               "q": 0.35, # Possibility of moving away from the previous vertex, how well you are exploring new places
               "print_logger": 10,  # printing rate
               "eps":0.0, # for GIN only
               }

In [47]:
params_list = {"model_type": "GCN",  # "GCN", "GAT", "GIN", "GraphSAGE"
               "n_graph_subsampling": 0, # the number of running graph subsampling each train graph data run subsampling 5 times: increasing graph data 5 times
               "graph_node_subsampling": True, # TRUE: removing node randomly to subsampling and augmentation of graph dataset \n'+
                # FALSE: removing edge randomly to subsampling and augmentation of graph dataset
               "graph_subsampling_rate": 0.2, # graph subsampling rate
               "dataset": "DD", 
               "pooling_type": "mean", 
               "seed": 42,
               "n_folds": 10, 
               "cuda": True, 
               "lr": [0.0001, 0.001], 
               "epochs": 1, 
               "weight_decay":5e-4,
               "batch_size": 32, 
               "dropout": 0, # dropout rate of layer
               "num_lay": [2, 3], 
               "num_agg_layer": 2, # the number of graph aggregation layers
               "hidden_agg_lay_size": [16, 32], # size of hidden graph aggregation layer
               "fc_hidden_size": 128, # size of fully-connected layer after readout
               "threads":10, # how many subprocesses to use for data loading
               "random_walk":True,
               "walk_length": 20, # walk length of random walk, 
               "num_walk": 10, # num of random walk
               "p": 0.65, # Possibility to return to the previous vertex, how well you navigate around
               "q": 0.35, # Possibility of moving away from the previous vertex, how well you are exploring new places
               "print_logger": 10,  # printing rate
               "eps":0.0, # for GIN only
               }

In [48]:
class grid: 
    
    def __init__(self, params_dict): 
        self.params_dict = params_dict

    def get_combinations(self):
        keys, values = zip(*self.params_dict.items())
        return [dict(zip(keys, v)) for v in itertools.product(*values)]

    def __repr__(self):
        return f"HyperparameterGrid({self.param_dict})"

In [49]:
class ModelSelection(): 
    
    def __init__(self, DATASET, data_split, grid, random_search=False, n_samples=10): 

        self.grid = grid
        self.DATASET = DATASET
        self.train_split = GraphDatasetSubset(self.DATASET.dataset.get_data(), data_split["train"])
        self.validation_split = GraphDatasetSubset(self.DATASET.dataset.get_data(), data_split["validation"])
        self.best_config = None
        self.random_search = random_search
        self.n_samples = n_samples

    def fit(self):
        dic = {"lr" : self.grid["lr"], "num_lay" : self.grid["num_lay"], "hidden_agg_lay_size" : self.grid["hidden_agg_lay_size"]}
        grille = grid(dic)
        all_combinations = grille.get_combinations()
        
        if self.random_search:
            combinations_to_try = random.sample(all_combinations, min(self.n_samples, len(all_combinations)))
        else:
            combinations_to_try = all_combinations

        for config in combinations_to_try:
            print(f"Training with configuration: {config}")
            params = self.grid
            params["lr"] = config["lr"]
            params["num_lay"] = config["num_lay"]
            params["hidden_agg_lay_size"] = config["hidden_agg_lay_size"]

            print(len(self.train_split))
            trainer = Train(params, data = self.train_split)
            trainer.fit()
            accuracy = trainer.evaluate(self.validation_split, graphDATA = True)

            print(f"Validation accuracy: {accuracy}")

            if self.best_config is None or accuracy > self.best_config["accuracy"]:
                self.best_config = {"config": params, "accuracy": accuracy}
        
    def get_best_config(self):
        if self.best_config is None:
            raise ValueError("No configurations have been evaluated.")
        return self.best_config["config"], self.best_config["accuracy"]


In [50]:
dic = {"lr" : params_list["lr"], "num_lay" : [1, 2], "hidden_agg_lay_size" : [16, 32, 64]}
grille = grid(dic)
grille.get_combinations()


[{'lr': 0.0001, 'num_lay': 1, 'hidden_agg_lay_size': 16},
 {'lr': 0.0001, 'num_lay': 1, 'hidden_agg_lay_size': 32},
 {'lr': 0.0001, 'num_lay': 1, 'hidden_agg_lay_size': 64},
 {'lr': 0.0001, 'num_lay': 2, 'hidden_agg_lay_size': 16},
 {'lr': 0.0001, 'num_lay': 2, 'hidden_agg_lay_size': 32},
 {'lr': 0.0001, 'num_lay': 2, 'hidden_agg_lay_size': 64},
 {'lr': 0.001, 'num_lay': 1, 'hidden_agg_lay_size': 16},
 {'lr': 0.001, 'num_lay': 1, 'hidden_agg_lay_size': 32},
 {'lr': 0.001, 'num_lay': 1, 'hidden_agg_lay_size': 64},
 {'lr': 0.001, 'num_lay': 2, 'hidden_agg_lay_size': 16},
 {'lr': 0.001, 'num_lay': 2, 'hidden_agg_lay_size': 32},
 {'lr': 0.001, 'num_lay': 2, 'hidden_agg_lay_size': 64}]

In [51]:
class Holdout(): 
    def __init__(self, DATASET, train_split, train_size=0.9): 
        self.DATASET = DATASET
        self.train_split = train_split
        self.train_size = train_size
        self.num_samples = len(self.train_split)
        self.train_indices = None

    def split(self):

        indices = np.arange(self.num_samples)
        np.random.shuffle(indices)
        self.train_indices = indices[:int(self.num_samples * self.train_size)]
       

    def get_splits(self):
        if self.train_indices is None :
            raise ValueError("Data has not been split yet.")
        return self.train_indices

In [52]:
class ModelAssessment(): 
    
    def __init__(self, DATASET, grid, Kfold=10, holdout=3): 
        self.DATASET = DATASET
        self.grid = grid
        self.Kfold = Kfold
        self.best_configs = None
        self.splits = DATASET.splits
        self.holdout = holdout
        self.fold_accuracy = []

    def assess(self):

        for i in tqdm(range(self.Kfold)):
            train_data_idxs = self.splits[i]["model_selection"][0]
            test_data_idxs = self.splits[i]["test"]
            test_data = GraphDatasetSubset(self.DATASET.dataset.get_data(), test_data_idxs)

            model_selection = ModelSelection(self.DATASET, train_data_idxs, self.grid)
            model_selection.fit()
            best_config, _ = model_selection.get_best_config()
            self.best_configs = best_config
            inner_accuracy = []

            for j in range(self.holdout): 
                
                train_holdout = Holdout(self.DATASET, train_data_idxs)
                train_holdout.split()
                train_indices = train_holdout.get_splits()
                
                train_data = GraphDatasetSubset(self.DATASET.dataset.get_data(), train_indices)
                model = Train(self.best_configs, train_data)
                model.fit()
                accuracy = model.evaluate(test_data, graphDATA = True)
                inner_accuracy.append(accuracy)
            
            self.fold_accuracy.append(np.mean(inner_accuracy))
        
        return np.mean(self.fold_accuracy), np.std(self.fold_accuracy)

In [53]:
idxs = IMDB.splits[0]["model_selection"][0]
train_data = GraphDatasetSubset(IMDB.dataset.get_data(), idxs["train"])
val_data = GraphDatasetSubset(IMDB.dataset.get_data(), idxs["validation"])

In [42]:
ModelAssessment__ = ModelAssessment(DD, params_list)

In [43]:
ModelAssessment__.assess()

  0%|          | 0/10 [00:00<?, ?it/s]

Training with configuration: {'lr': 0.0001, 'num_lay': 2, 'hidden_agg_lay_size': 16}
954
taille de x_dataset: 954 et taille de y_dataset : 954
N trainable parameters: 4146


Accuracy: 559/954 (58.60%)

Best Accuracy: 58.60%
Average training time per epoch: 0.04 seconds
Accuracy: 62/106 (58.49%)

Validation accuracy: 58.490566037735846
Training with configuration: {'lr': 0.0001, 'num_lay': 2, 'hidden_agg_lay_size': 32}
954
taille de x_dataset: 954 et taille de y_dataset : 954
N trainable parameters: 8418


Accuracy: 559/954 (58.60%)

Best Accuracy: 58.60%
Average training time per epoch: 0.06 seconds
Accuracy: 62/106 (58.49%)

Validation accuracy: 58.490566037735846
Training with configuration: {'lr': 0.0001, 'num_lay': 3, 'hidden_agg_lay_size': 16}
954
taille de x_dataset: 954 et taille de y_dataset : 954
N trainable parameters: 4146


Accuracy: 559/954 (58.60%)

Best Accuracy: 58.60%
Average training time per epoch: 0.04 seconds
Accuracy: 62/106 (58.49%)

Validation accuracy: 58.490566037735846
Training with configuration: {'lr': 0.0001, 'num_lay': 3, 'hidden_agg_lay_size': 32}
954
taille de x_dataset: 954 et taille de y_dataset : 954
N trainable parameters: 8418


...

Accuracy: 559/954 (58.60%)

Best Accuracy: 58.60%
Average training time per epoch: 0.04 seconds
Accuracy: 62/106 (58.49%)

Validation accuracy: 58.490566037735846
Training with configuration: {'lr': 0.001, 'num_lay': 3, 'hidden_agg_lay_size': 32}
954
taille de x_dataset: 954 et taille de y_dataset : 954
N trainable parameters: 8418


Accuracy: 559/954 (58.60%)

Best Accuracy: 58.60%
Average training time per epoch: 0.04 seconds


  0%|          | 0/10 [10:40<?, ?it/s]

Accuracy: 62/106 (58.49%)

Validation accuracy: 58.490566037735846


KeyError: 'dataset'

On tente avec une configuration random pour voir si l'entraînement se lance

---------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
data = GraphDatasetSubset(DD.dataset.get_data(), idxs["train"])
adj, features = get_adjacency_and_features(data[0])
print("Adjency matrix:\n")
print(adj)
print("Feature vector: \n")
print(features)

Adjency matrix:

tensor([[0.2000, 0.1826, 0.2236,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1667, 0.2041,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])
Feature vector: 

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [ ]:
class Train:
    def __init__(self, params, dataset):
      self.params = params 
      self.x_dataset, self.y_dataset = dataset.dataset.get_data(), dataset.dataset.get_targets()

      if self.params["cuda"] and torch.cuda.is_available():
          self.device = "cuda:0"
      else: 
          self.device = "cpu"

      self.model = self.get_model()
      self.loss_fn = F.cross_entropy 

    
    def get_model(self):
        
        #adjacency, features = get_adjacency_and_features(self.dataset[0])
      
        if self.params["model_type"] == 'GCN':
          
            model = GCN(n_feat = self.x_dataset[0].x.shape[1],  # [N, F] → F
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params['pooling_type'],
                    device=self.device).to(self.device)
            
        elif self.params["model_type"] == 'GAT':
            
            model = GAT(n_feat=self.x_dataset[0].x.shape[1],
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params['pooling_type'],
                    device=self.device).to(self.device)
            
        elif self.params["model_type"] == 'GraphDenseNet':
            
            model = GraphDenseNet(n_feat=self.x_dataset[0].x.shape[1],
                    n_class=2,
                    n_layer=self.params['num_agg_layer'],
                    agg_hidden=self.params['hidden_agg_lay_size'],
                    fc_hidden=self.params['fc_hidden_size'],
                    dropout=self.params['dropout'],
                    pool_type=self.params["pooling_type"],
                    device=self.device).to(self.device)
            
        return model


    def loaders_train_test_setup(self):

        loader = torch.utils.data.DataLoader(range(len(self.x_dataset)),
                            batch_size=self.params["batch_size"],
                            shuffle=True,
                            num_workers=0,
                            pin_memory=True,
                            drop_last=False,
                            collate_fn=lambda x: x)  # x = liste d'indices


      
      # Total trainable param
        c = 0
        for p in filter(lambda p: p.requires_grad, self.model.parameters()):
            c += p.numel()
        print('N trainable parameters:', c)

        optimizer = optim.Adam(
                        filter(lambda p: p.requires_grad, self.model.parameters()),
                        lr=self.params["lr"],
                        weight_decay=self.params["weight_decay"],
                        betas=(0.5, 0.999))
    
        scheduler = lr_scheduler.MultiStepLR(optimizer, [20, 30], gamma=0.1)
            
        return loader, optimizer, scheduler

    def train(self, train_loader, optimizer, scheduler, epoch):
        total_time_iter = 0
        self.model.train()
        start = time.time()
        train_loss, n_samples = 0, 0

        for batch_idx, data_batch in enumerate(train_loader):
            # === Préparer le batch manuellement ===
            x_batch, adj_batch, y_batch, sizes = create_batch_from_loader(data_batch, self.x_dataset, self.y_dataset, self.device)

            # === Forward + backward ===
            optimizer.zero_grad()
            output = self.model(x_batch, adj_batch)
            loss = self.loss_fn(output, y_batch)

            loss.backward()
            optimizer.step()

            # === Stat tracking ===
            time_iter = time.time() - start
            total_time_iter += time_iter
            train_loss += loss.item() * len(y_batch)
            n_samples += len(y_batch)

            if batch_idx % self.params["print_logger"] == 0 or batch_idx == len(train_loader) - 1:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} (avg: {:.6f}) \tsec/iter: {:.4f}'.format(
                    epoch, n_samples, len(train_loader.dataset),
                    100. * (batch_idx + 1) / len(train_loader), loss.item(), train_loss / n_samples, time_iter / (batch_idx + 1)
                ))

        scheduler.step()
        return total_time_iter / (len(train_loader) + 1)


    def evaluate(self, test_loader):
        self.model.eval()
        correct, n_samples = 0, 0

        for batch_idx, data in enumerate(test_loader):
            for i in range(len(data)):
                data[i] = data[i].to(self.device)
            
            output = self.model(data)
            pred = output.detach().cpu().max(1, keepdim=True)[1]
            correct += pred.eq(data[4].detach().cpu().view_as(pred)).sum().item()
            n_samples += len(output)

        acc = 100. * correct / n_samples
        print('Test set (epoch {}): Accuracy: {}/{} ({:.2f}%)\n'.format(self.params["epochs"], correct, n_samples, acc))
        
        return acc
         
              
    def fit(self): 
        loader, optimizer, scheduler = self.loaders_train_test_setup()
        total_time = 0

        best_acc = 0
        patience_counter = 0
        patience = self.params.get("early_stopping_patience", 5)

        for epoch in tqdm(range(self.params["epochs"]), desc="Epochs", position=1, leave=False):
            total_time_iter = self.train(loader, optimizer, scheduler, epoch)
            total_time += total_time_iter
            acc = self.evaluate(loader)

            if acc > best_acc:
                best_acc = acc
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

        print(f'Best Accuracy: {best_acc:.2f}%')
        print(f'Average training time per epoch: {total_time / (epoch + 1):.2f} seconds')

        result_list = [self.params["dataset"], self.params["dataset"]]
        result_list.append(best_acc)



In [ ]:
trainer = Train(params_list, IMDB)

In [ ]:
features = features.reshape(-1, features.size(1))
num_agg_layer, hidden_agg_layer_size = 2, 64
weights = nn.Parameter(torch.FloatTensor(num_agg_layer, hidden_agg_layer_size))
features = torch.mm(features, weights)
print(features.shape)
print(adj.shape)

NameError: name 'nn' is not defined

In [ ]:
model = trainer.model
loader = trainer.dataset
x_batch, adj_batch, y_batch = create_batch_from_loader(loader)
output = model(x_batch, adj_batch)

AttributeError: 'GraphData' object has no attribute 'get_targets'

In [ ]:
trainer.fit()

N trainable parameters: 12866


support shape: torch.Size([650, 64])
x shape: torch.Size([650, 1])
adj shape: torch.Size([10726, 10726])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10726x10726 and 650x64)